In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import optuna

In [2]:
# Model 1
df = pd.read_csv("data/train_folds.csv")
df_test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
numerical_cols = [col for col in useful_features if col.startswith("cont")]
df_test = df_test[useful_features]

for col in numerical_cols:
    df[col] = np.log1p(df[col])
    df_test[col] = np.log1p(df_test[col])

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]


In [2]:
def run(trial):
    fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])

    model = XGBRegressor(
        random_state=42,
        tree_method='gpu_hist',
        gpu_id=1,
        predictor="gpu_predictor",
        n_estimators=7000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [4]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=100) #100

[I 2021-08-29 17:16:26,798] A new study created in memory with name: no-name-7dd346c2-e927-4a92-9cdf-0b59ce19e471


[17:16:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.64445
[1000]	validation_0-rmse:0.72663
[2000]	validation_0-rmse:0.72255
[3000]	validation_0-rmse:0.72050
[4000]	validation_0-rmse:0.71928
[5000]	validation_0-rmse:0.71856
[6000]	validation_0-rmse:0.71809
[6999]	validation_0-rmse:0.71783


[I 2021-08-29 17:17:28,191] Trial 0 finished with value: 0.7178241183314773 and parameters: {'learning_rate': 0.01756937523162302, 'reg_lambda': 1.0853151506759414e-07, 'reg_alpha': 2.5160929834612467, 'subsample': 0.6362270243609827, 'colsample_bytree': 0.3333421958988413, 'max_depth': 3}. Best is trial 0 with value: 0.7178241183314773.


[17:17:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.68761
[1000]	validation_0-rmse:0.72182
[2000]	validation_0-rmse:0.71971
[3000]	validation_0-rmse:0.71926
[3539]	validation_0-rmse:0.71927


[I 2021-08-29 17:18:54,935] Trial 1 finished with value: 0.719203740606503 and parameters: {'learning_rate': 0.011969126061443562, 'reg_lambda': 0.00908222386310185, 'reg_alpha': 0.25781199761018764, 'subsample': 0.9290904339770187, 'colsample_bytree': 0.4779906301858352, 'max_depth': 7}. Best is trial 0 with value: 0.7178241183314773.


...

[I 2021-08-29 18:45:02,975] Trial 97 finished with value: 0.71708954303124 and parameters: {'learning_rate': 0.024439210487440542, 'reg_lambda': 18.77100771245707, 'reg_alpha': 2.670030252295931, 'subsample': 0.6977154819880925, 'colsample_bytree': 0.1380321161124814, 'max_depth': 4}. Best is trial 51 with value: 0.7169064837392827.


[18:45:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.62123
[1000]	validation_0-rmse:0.72310
[2000]	validation_0-rmse:0.71955
[3000]	validation_0-rmse:0.71824
[4000]	validation_0-rmse:0.71763
[5000]	validation_0-rmse:0.71738
[5261]	validation_0-rmse:0.71740


[I 2021-08-29 18:45:54,847] Trial 98 finished with value: 0.7173808135080316 and parameters: {'learning_rate': 0.020587555581724565, 'reg_lambda': 6.623262643487766, 'reg_alpha': 4.2072925946675664e-05, 'subsample': 0.5481507083493457, 'colsample_bytree': 0.2029470107251543, 'max_depth': 4}. Best is trial 51 with value: 0.7169064837392827.


[18:45:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.56913
[1000]	validation_0-rmse:0.72333
[2000]	validation_0-rmse:0.72008
[3000]	validation_0-rmse:0.71898
[4000]	validation_0-rmse:0.71856
[5000]	validation_0-rmse:0.71839
[5822]	validation_0-rmse:0.71836


[I 2021-08-29 18:47:04,368] Trial 99 finished with value: 0.7183400930175847 and parameters: {'learning_rate': 0.027361891720247507, 'reg_lambda': 80.24704110944548, 'reg_alpha': 63.679263739724455, 'subsample': 0.7188690417044336, 'colsample_bytree': 0.5024822510206274, 'max_depth': 4}. Best is trial 51 with value: 0.7169064837392827.


In [5]:
study.best_params

{'learning_rate': 0.0342032910688893,
 'reg_lambda': 16.984336234349644,
 'reg_alpha': 13.2209270507238,
 'subsample': 0.7396101488182523,
 'colsample_bytree': 0.13653907086890005,
 'max_depth': 3}

In [7]:
# Model 2
df = pd.read_csv("data/train_folds.csv")
df_test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=100) #100

[I 2021-08-29 18:47:44,719] A new study created in memory with name: no-name-7acf527b-9e92-426f-8318-4ce3028973d7


[18:47:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.63239
[1000]	validation_0-rmse:0.72154
[2000]	validation_0-rmse:0.72028
[2305]	validation_0-rmse:0.72037


[I 2021-08-29 18:48:21,683] Trial 0 finished with value: 0.7202673872752845 and parameters: {'learning_rate': 0.0191379813907593, 'reg_lambda': 1.4363127879918497e-08, 'reg_alpha': 0.00041415689595605197, 'subsample': 0.4170488901687981, 'colsample_bytree': 0.8690280656001025, 'max_depth': 6}. Best is trial 0 with value: 0.7202673872752845.


[18:48:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.36516
[1000]	validation_0-rmse:0.71982
[1951]	validation_0-rmse:0.71943


[I 2021-08-29 18:48:48,109] Trial 1 finished with value: 0.7193494439165233 and parameters: {'learning_rate': 0.05384256063858188, 'reg_lambda': 8.776323950846218e-07, 'reg_alpha': 48.421734302412716, 'subsample': 0.5053790749130518, 'colsample_bytree': 0.8462785533989301, 'max_depth': 5}. Best is trial 1 with value: 0.7193494439165233.


...

[I 2021-08-29 20:30:40,155] Trial 97 finished with value: 0.7173690207110963 and parameters: {'learning_rate': 0.018673727237094442, 'reg_lambda': 5.33214670254417, 'reg_alpha': 3.692281276550072, 'subsample': 0.646854725143019, 'colsample_bytree': 0.14328177227517755, 'max_depth': 6}. Best is trial 73 with value: 0.7169119261340184.


[20:30:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.69334
[1000]	validation_0-rmse:0.72787
[2000]	validation_0-rmse:0.72107
[3000]	validation_0-rmse:0.71868
[4000]	validation_0-rmse:0.71778
[5000]	validation_0-rmse:0.71737
[6000]	validation_0-rmse:0.71719
[6999]	validation_0-rmse:0.71712


[I 2021-08-29 20:32:28,964] Trial 98 finished with value: 0.7171147064675906 and parameters: {'learning_rate': 0.011228487579705387, 'reg_lambda': 12.360183215832858, 'reg_alpha': 10.554912050577249, 'subsample': 0.7964916478773698, 'colsample_bytree': 0.11772466556467015, 'max_depth': 6}. Best is trial 73 with value: 0.7169119261340184.


[20:32:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.69265
[1000]	validation_0-rmse:0.72453
[2000]	validation_0-rmse:0.71982
[3000]	validation_0-rmse:0.71831
[4000]	validation_0-rmse:0.71762
[5000]	validation_0-rmse:0.71730
[6000]	validation_0-rmse:0.71721
[6141]	validation_0-rmse:0.71723


[I 2021-08-29 20:34:11,483] Trial 99 finished with value: 0.7172095737207993 and parameters: {'learning_rate': 0.011317078535432935, 'reg_lambda': 10.046690254077859, 'reg_alpha': 11.950519329184143, 'subsample': 0.8080841261613334, 'colsample_bytree': 0.19953454892298766, 'max_depth': 6}. Best is trial 73 with value: 0.7169119261340184.


In [8]:
study.best_params

{'learning_rate': 0.015603699193600313,
 'reg_lambda': 0.001158068570366239,
 'reg_alpha': 16.448429406371936,
 'subsample': 0.5948693703077814,
 'colsample_bytree': 0.12425047105688761,
 'max_depth': 5}

# Model 3 Test

In [8]:
df = pd.read_csv("data/train_folds.csv")
df_test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

df1 = pd.read_csv("train_pred_1.csv")
df1.columns = ["id", "pred_1"]
df2 = pd.read_csv("train_pred_2.csv")
df2.columns = ["id", "pred_2"]
df3 = pd.read_csv("train_pred_3.csv")


df_test1 = pd.read_csv("test_pred_1.csv")
df_test1.columns = ["id", "pred_1"]
df_test2 = pd.read_csv("test_pred_2.csv")
df_test2.columns = ["id", "pred_2"]
df_test3 = pd.read_csv("test_pred_3.csv")


df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")


df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")


In [9]:
def run(trial):
    fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

    model = XGBRegressor(
        random_state=42,
        tree_method='gpu_hist',
        gpu_id=1,
        predictor="gpu_predictor",
        n_estimators=7000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [ ]:
useful_features = ["pred_1", "pred_2"]
df_test = df_test[useful_features]

study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=700) #100

[I 2021-08-30 17:48:11,161] A new study created in memory with name: no-name-b1b60ac7-14c1-4d81-bd2d-ba36c4d25517


[17:48:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.25550
[426]	validation_0-rmse:0.71720


[I 2021-08-30 17:48:12,770] Trial 0 finished with value: 0.7171320126150608 and parameters: {'learning_rate': 0.06814068146333314, 'reg_lambda': 57.05917533101725, 'reg_alpha': 1.6945206004814216e-07, 'subsample': 0.40290355172397874, 'colsample_bytree': 0.552718577129548, 'max_depth': 3}. Best is trial 0 with value: 0.7171320126150608.


[17:48:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.53873
[718]	validation_0-rmse:0.71716


[I 2021-08-30 17:48:14,971] Trial 1 finished with value: 0.7171374210694197 and parameters: {'learning_rate': 0.031274082647908076, 'reg_lambda': 3.099278734687273e-08, 'reg_alpha': 1.4729489977175496, 'subsample': 0.6361136543980302, 'colsample_bytree': 0.45979571179862166, 'max_depth': 2}. Best is trial 0 with value: 0.7171320126150608.


...

[18:02:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:6.70412
[414]	validation_0-rmse:0.71712


[I 2021-08-30 18:02:09,341] Trial 451 finished with value: 0.7170499786317348 and parameters: {'learning_rate': 0.13966636719081169, 'reg_lambda': 4.489036275592246e-05, 'reg_alpha': 2.4352476507467962e-08, 'subsample': 0.14136896842921276, 'colsample_bytree': 0.37196352342322, 'max_depth': 1}. Best is trial 321 with value: 0.7170332013860602.


[18:02:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:6.70658
[445]	validation_0-rmse:0.71714


[I 2021-08-30 18:02:10,831] Trial 452 finished with value: 0.7170918959249646 and parameters: {'learning_rate': 0.13935038691786286, 'reg_lambda': 4.921830422695001e-05, 'reg_alpha': 2.530784685165321e-08, 'subsample': 0.1928000420610475, 'colsample_bytree': 0.2029753726603168, 'max_depth': 1}. Best is trial 321 with value: 0.7170332013860602.


[18:02:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:6.61381


In [6]:
study.best_params

{'learning_rate': 0.019899422921897673,
 'reg_lambda': 2.0215144838140033,
 'reg_alpha': 1.851725803396807e-06,
 'subsample': 0.12135463036938088,
 'colsample_bytree': 0.39726269844915346,
 'max_depth': 2}